# Visualizing EOD data in Flowmap using the Unitrip format

This notebook shows how to use the module created to convert Foursquare data to the Unitrip format. Then, from the format created, the data can be converted so that it can be visualized in flowmap.blue

## Preamble

In [1]:
import seaborn as sns
import pandas as pd
import geopandas as gpd

import sys
import os.path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

%matplotlib inline
sns.set(context='notebook', font='Lucida Sans Unicode', style='white', palette='plasma')

Import EOD data: Trips

In [2]:
eod_trips = pd.read_csv('../data/other_format/EOD_STGO/viajes.csv', sep=';')
eod_trips = eod_trips[['Persona', 'Viaje', 'OrigenCoordX', 'OrigenCoordY', 'DestinoCoordX', 'DestinoCoordY', 'HoraIni', 'HoraFin']]
eod_trips.head()

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin
0,17343102,1734310202,"335208,7188",6288387,"338812,3125",6292391,22:30,23:40
1,17344101,1734410101,"338536,4375",6291928,"354267,3438",6302297,13:00,14:45
2,17344101,1734410102,"354267,3438",6302297,"338536,4375",6291928,22:00,23:30
3,17344103,1734410301,"338536,4375",6291928,"350841,6563",6297212,9:00,9:55
4,17344103,1734410302,"350841,6563",6297212,"338536,4375",6291928,19:00,21:30


The module is used to convert the format to unitrip.

In [5]:
from unitrip.format_conversion.eod_converter import eod_to_unitrip

eod_to_unitrip(eod_trips, '../data/unified_format/santiago_eod_unitrip.parquet', h3_res=12)

Fixing origin and destination time formats ...
Calculating h3 cells to origin and destination coords ...
Dataframe ready to store
File stored at ../data/unified_format/santiago_eod_unitrip.parquet


## Let's aggregate trips and create flows in unitrip format (unitrip -> uniflow)

Load the parquet to a dataframe with the unitrip data

In [6]:
santiago_unitrip = pd.read_parquet('../data/unified_format/santiago_eod_unitrip.parquet', columns=['user_id', 'o_h3_cell', 'd_h3_cell'])
santiago_unitrip.head()

,user_id,o_h3_cell,d_h3_cell
trip_id,,,
0,17343102,8cb2c540d0a89ff,8cb2c5429b60dff
1,17344101,8cb2c5429ae5dff,8cb2c519a994dff
2,17344101,8cb2c519a994dff,8cb2c5429ae5dff
3,17344103,8cb2c5429ae5dff,8cb2c554db113ff
4,17344103,8cb2c554db113ff,8cb2c5429ae5dff


The aggregation will be performed using h3 cells, with a specific level of resolution and filtering with a minimum number of trips per flow

In [7]:
from unitrip.unified_format.trip_to_flow import unitrip_to_uniflow

unitrip_to_uniflow(santiago_unitrip, '../data/unified_format/santiago_eod_uniflow.parquet', flow_res=7, minimun_trips=8)

Set the h3 cell columns to a parent resolution ...
Trips from one cell to the same cell are deleted, and the same OD trips made by a user are grouped together. ...
There are 72707 unique trips
Aggregating trips of the same OD and with a minimun number of trips ...
There are 1914 flows
File stored at ../data/unified_format/santiago_eod_uniflow.parquet


## Let's create the flowmap.blue input from flows in unitrip format (uniflow)

Load the parquet to a dataframe with the Uniflow data

In [8]:
santiago_uniflow = pd.read_parquet('../data/unified_format/santiago_eod_uniflow.parquet')
santiago_uniflow.head()

,o_h3_cell,d_h3_cell,count
43,87b2c0853ffffff,87b2c085effffff,84
47,87b2c0853ffffff,87b2c08ecffffff,10
48,87b2c0853ffffff,87b2c08edffffff,61
77,87b2c085cffffff,87b2c085effffff,10
88,87b2c085effffff,87b2c0853ffffff,83


The module is used to convert flows (in unitrip format) to flowmap input

In [9]:
from unitrip.use_cases.flowmap_blue.flow_to_flowmap import generate_flows_locations

generate_flows_locations(santiago_uniflow, '../data/use_cases/flowmap/santiago_eod_flows.csv', '../data/use_cases/flowmap/santiago_eod_locations.csv')

flows file stored at ../data/use_cases/flowmap/santiago_eod_flows.csv
locations file stored at ../data/use_cases/flowmap/santiago_eod_locations.csv


Once these two files have been uploaded to flowmap.blue, the visualization can be seen [HERE](https://www.flowmap.blue/1oTll7OJe8yTegiUeLZgQ-lZR6fTmrEIu5OF52C_1IKk?v=-33.479061%2C-70.660043%2C10.77%2C0%2C0&a=0&as=1&b=1&bo=75&c=0&ca=1&d=1&fe=1&lt=1&lfm=ALL&col=DarkMint&f=50)